In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)

In [2]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    context_schema=ColourContext  
)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='5e32f8f2-f924-4ede-91ed-bcad19517c74'),
              AIMessage(content='I’m not sure what your favorite colour is—could you let me know?', additional_kwargs={'reasoning_content': 'The user asks "What is my favourite colour?" There\'s no prior context. We don\'t know. According to policy, we must respond that we don\'t know. We can ask clarifying question or say we don\'t know. The user might be expecting a guess, but we shouldn\'t guess. We can ask them to tell us. So answer: I don\'t know, could you tell me?'}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 77, 'total_tokens': 179, 'completion_time': 0.212338761, 'completion_tokens_details': {'reasoning_tokens': 77}, 'prompt_time': 0.003462607, 'prompt_tokens_details': None, 'queue_time': 0.004756993, 'total_time': 0.215801368}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp

## Accessing Context

In [11]:
from typing import Annotated
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

@tool
def get_favourite_colour(config: RunnableConfig) -> str:
    """Get the favourite colour of the user"""
    # Accessing the context passed via the 'configurable' field
    context = config.get("configurable", {}).get("context")
    if context:
        return context.favourite_colour
    return "I don't know the favourite colour."

@tool
def get_least_favourite_colour(config: RunnableConfig) -> str:
    """Get the least favourite colour of the user"""
    context = config.get("configurable", {}).get("context")
    if context:
        return context.least_favourite_colour
    return "I don't know the least favourite colour."

In [12]:
agent = create_agent(
    model=model,
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext
)

In [ ]:
config = {
    "configurable": {
        "thread_id": "session_123",
        "context": ColourContext(
            favourite_colour="yellow", 
            least_favourite_colour="beige"
        )
    }
}

In [ ]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    config=config
)

pprint(response)

In [14]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my least favourite colour?")]},
    config=config
)

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='344373a6-6cc2-4c35-8f2c-15ce9c265217'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to get favourite colour via function. Use get_favourite_colour.', 'tool_calls': [{'id': 'fc_11fe9e59-1bb5-4b0e-881e-b8ba748cba84', 'function': {'arguments': '{}', 'name': 'get_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 149, 'total_tokens': 186, 'completion_time': 0.076875658, 'completion_tokens_details': {'reasoning_tokens': 16}, 'prompt_time': 0.005924823, 'prompt_tokens_details': None, 'queue_time': 0.004751634, 'total_time': 0.082800481}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_309cec8e88', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c447f-3fde-7593-a9ca-6f96559a52fd-0', too